In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
import dask.multiprocessing

# 1. Preprocessing data
## Load the raw data file

In [28]:

%time ddf = dd.read_csv("../data/raw/features_file_ml7.csv", sample=2560000)

# df = pd.read_csv("data/raw/features_file_ml7.csv")

%time df = ddf.compute(scheduler="processes")

CPU times: user 1min 14s, sys: 500 ms, total: 1min 15s
Wall time: 1min 15s
CPU times: user 1min, sys: 57.9 s, total: 1min 58s
Wall time: 2min 52s


### Extract all each features with labels

In [4]:
# opcodes
%time df_opcodes = df.filter(regex="^OPCODE*|^label$")

CPU times: user 80 ms, sys: 12 ms, total: 92 ms
Wall time: 91.2 ms


In [5]:
#permissions
%time df_permissions = df.filter(regex="^PERMISSION|^label$")

CPU times: user 344 ms, sys: 192 ms, total: 536 ms
Wall time: 526 ms


In [29]:
#api packages
%time df_api_packages = df.filter(regex="^APIPACKA|^label$")

CPU times: user 112 ms, sys: 36 ms, total: 148 ms
Wall time: 145 ms


In [68]:
# api calls
%time df_api_calls = df.filter(regex="^APICALL|^label$")

CPU times: user 180 ms, sys: 152 ms, total: 332 ms
Wall time: 790 ms


In [31]:
# intent receivers
%time df_intent_receivers = df.filter(regex="^RECEIVER|^label$")

CPU times: user 312 ms, sys: 320 ms, total: 632 ms
Wall time: 622 ms


In [32]:
# intent services
%time df_intent_services = df.filter(regex="^SERVICE|^label$")

CPU times: user 248 ms, sys: 224 ms, total: 472 ms
Wall time: 471 ms


In [33]:
# intent activities
%time df_intent_activity = df.filter(regex="^ACTIVITY|^label$")

CPU times: user 344 ms, sys: 292 ms, total: 636 ms
Wall time: 631 ms


In [26]:
# system calls
%time df_system = df.filter(regex="^SYSTEM|^label$")

CPU times: user 68 ms, sys: 20 ms, total: 88 ms
Wall time: 85.2 ms


In [69]:
df_api_calls.describe()

,APICALL-java.security.cert.Certificate,APICALL-android.view.SubMenu,APICALL-android.media.audiofx.Virtualizer,APICALL-android.support.v7.widget.helper.ItemTouchHelper,APICALL-android.net.RouteInfo,APICALL-android.bluetooth.BluetoothGattCallback,APICALL-android.support.percent.PercentLayoutHelper,APICALL-android.print.PageRange,APICALL-java.net.InterfaceAddress,APICALL-android.support.v7.widget.DividerItemDecoration,...,APICALL-android.speech,APICALL-android.content.ClipDescription,APICALL-android.support.v4.view.accessibility.AccessibilityManagerCompat,APICALL-java.net.InetAddress,APICALL-android.support.multidex.MultiDexApplication,APICALL-android.widget.MediaController,APICALL-android.support.v4.widget.ExploreByTouchHelper,APICALL-android.content.ContentResolver,APICALL-org.xmlpull.v1.XmlPullParser,APICALL-org.xml.sax.ext.DeclHandler
count,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.00000,22000.000000,22000.000000,...,22000.0,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000
mean,0.292545,2.596636,0.005591,5.097636,0.002636,0.003182,0.030364,0.00200,0.008500,0.001227,...,0.0,0.635682,1.367000,3.766773,0.025727,1.629273,12.803818,12.108273,31.229045,0.036455
std,2.113503,4.354389,0.230056,21.344695,0.179884,0.151927,0.601365,0.13617,0.222124,0.060667,...,0.0,0.950356,1.499259,8.454665,0.230167,4.756713,15.146042,25.144656,92.923261,0.383697
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,7.000000,9.000000,0.000000
75%,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,2.000000,3.000000,4.000000,0.000000,1.000000,30.000000,14.000000,16.000000,0.000000
max,50.000000,58.000000,16.000000,101.000000,13.000000,12.000000,12.000000,18.00000,19.000000,3.000000,...,0.0,15.000000,65.000000,212.000000,5.000000,102.000000,62.000000,1015.000000,2014.000000,10.000000


In [28]:
# save all features in csv data files
df_system.to_csv("../data/interim/system_commands.csv")

In [24]:
%%time
print("saving 1 ...")
df_intent_services.to_csv("../data/interim/intent_services.csv")
print("saving 2 ... ")
df_intent_activity.to_csv("../data/interim/intent_activities.csv")
print("saving 3 ...")
df_intent_receivers.to_csv("../data/interim/intent_receivers.csv")
print("saving 4 ... ")
df_api_packages.to_csv("../data/interim/api_packages.csv")

saving 1 ...
saving 2 ... 
saving 3 ...
saving 4 ... 
CPU times: user 11min 28s, sys: 8.9 s, total: 11min 37s
Wall time: 11min 37s


In [ ]:
df.loc[:,~df.columns.str.startwith("SYSTEM")]

In [29]:
mw_df = df[df["label"]=="MW"]
bw_df = df[df["label"]=="BW"]

In [31]:
#creating small datasets to test algorithms
from sklearn.model_selection import train_test_split

_, test_df = train_test_split(df, train_size=0.6)


In [33]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8800 entries, 682 to 560
Columns: 25999 entries, label to FLOWDROID-NETWORK_INFORMATION-NETWORK_INFORMATION
dtypes: int64(25998), object(1)
memory usage: 1.7+ GB


In [36]:
test_df = test_df.loc[:,~df.columns.str.startswith("FLOWDROID")]

In [37]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8800 entries, 682 to 560
Columns: 25038 entries, label to APIPACKAGE-android.app.job
dtypes: int64(25037), object(1)
memory usage: 1.6+ GB


In [38]:
test_df.to_csv("../data/interim/test/test_df.csv")

# 2. EDA (Exploratory Data Analysis)
## 2.1. Opcodes

In [3]:
#load data
df_opcodes = pd.read_csv("../data/interim/opcodes.csv")

In [11]:
df_opcodes = df_opcodes.iloc[:,1:]
df_opcodes.head()

,label,OPCODE-rem-float/2addr,OPCODE-rem-int/2addr,OPCODE-array-length,OPCODE-div-int/lit8,OPCODE-sub-float/2addr,OPCODE-and-int/lit8,OPCODE-and-int,OPCODE-int-to-long,OPCODE-move-wide/from16,...,OPCODE-new-instance,OPCODE-packed-switch,OPCODE-mul-int,OPCODE-shl-int,OPCODE-ushr-long/2addr,OPCODE-iget-object,OPCODE-or-int/lit16,OPCODE-const-wide,OPCODE-iput-wide,OPCODE-or-int/2addr
0,BW,3,4,674,61,38,109,10,94,45,...,7393,289,13,8,10,49439,4,45,284,1562
1,BW,2,1,143,84,39,164,26,28,32,...,2097,94,17,15,1,9363,5,28,88,64
2,BW,2,1,143,84,39,164,26,28,32,...,2097,94,17,15,1,9363,5,28,88,64
3,BW,1,0,546,33,20,73,9,40,39,...,5292,238,11,7,6,45389,4,34,255,1550
4,MW,0,0,43,6,2,11,0,9,32,...,852,7,0,0,0,1076,0,0,4,0


In [4]:
#split dataframe to benign apks and malware apks
df_bw_opcodes = df_opcodes[df_opcodes["label"] == ''"BW"]
df_mw_opcodes = df_opcodes[df_opcodes["label"] == "MW"]

### 10 most frequent opcodes found in smali code obtained for each application in the malware and benignware sets
#### Benign apks

In [57]:
%time df_bw_opcodes.where(df_bw_opcodes == 0, 1).sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_bw_opcodes))

CPU times: user 188 ms, sys: 44 ms, total: 232 ms
Wall time: 230 ms


OPCODE-return-void           1.000000
OPCODE-invoke-direct         1.000000
OPCODE-invoke-virtual        0.999909
OPCODE-invoke-super          0.999909
OPCODE-move-result-object    0.999727
OPCODE-invoke-static         0.999636
OPCODE-new-instance          0.999636
OPCODE-goto                  0.998818
OPCODE-const/4               0.998727
OPCODE-move-result           0.998727
dtype: float64

#### Malware apks

In [58]:
%time df_mw_opcodes.where(df_mw_opcodes == 0, 1).sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_mw_opcodes))

CPU times: user 268 ms, sys: 84 ms, total: 352 ms
Wall time: 328 ms


OPCODE-return-void           1.000000
OPCODE-invoke-direct         1.000000
OPCODE-new-instance          0.999545
OPCODE-invoke-virtual        0.999545
OPCODE-invoke-static         0.999545
OPCODE-move-result-object    0.999455
OPCODE-const/4               0.999364
OPCODE-move-result           0.999182
OPCODE-goto                  0.998818
OPCODE-if-eqz                0.997818
dtype: float64

In [48]:
%time df_mw_opcodes.sum(numeric_only=True).sort_values(ascending=False)[:10]

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 13.2 ms


OPCODE-invoke-virtual        197290846
OPCODE-iget-object           170801426
OPCODE-move-result-object    134955242
OPCODE-const/4                74287151
OPCODE-invoke-static          70909221
OPCODE-const-string           70571788
OPCODE-invoke-direct          68208422
OPCODE-iget                   67726925
OPCODE-move-result            59481415
OPCODE-return-void            49572934
dtype: int64

## 2.2. Permissions 

In [5]:
#load_data
df_permissions = pd.read_csv("../data/interim/permissions.csv", )

In [10]:
df_permissions = df_permissions.iloc[:,1:]
df_permissions.head()

,label,PERMISSION-com.amazon.CONTENT_PROVIDER_ACCESS,PERMISSION-com.mducrad.taskmanagerapp.permission.C2D_MESSAGE,PERMISSION-com.mywavia.poker.android.permission.C2D_MESSAGE,PERMISSION-android.permission.INTERNET,PERMISSION-com.yandex.promolib.permission.ACCESS_START_SVC,PERMISSION-com.android.buycott.permission.C2D_MESSAGE,PERMISSION-com.triactivemedia.familytree.permission.C2D_MESSAGE,PERMISSION-air.hdandv.hashgab.permission.C2D_MESSAGE,PERMISSION-com.Settings.widget.permission.SettingsAppWidgetProvider,...,PERMISSION-com.android.vending.INSTALL_REFERRER,PERMISSION-com.sarkarinaukrijobs.permission.C2D_MESSAGE,PERMISSION-com.apus.hola.launcher.permission.READ_SETTINGS,PERMISSION-com.kddi.market.permission.USE_ALML,PERMISSION-com.vivek.ayyada.permission.C2D_MESSAGE,PERMISSION-appsw7x.cristiantg.buspucelapro.permission.MAPS_RECEIVE,PERMISSION-com.triactivemedia.otaku.permission.C2D_MESSAGE,PERMISSION-com.uob.makeourmark.permission.C2D_MESSAGE,PERMISSION-com.mode.mwalimukenya.ui.permission.MAPS_RECEIVE,PERMISSION-com.protonge.packs.permission.C2D_MESSAGE
0,BW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#split dataframe to benign apks and malware apks
df_bw_permissions = df_permissions[df_permissions["label"] == ''"BW"]
df_mw_permissions = df_permissions[df_permissions["label"] == "MW"]

### 10 most frequent permissions found in smali code obtained for each application in the malware and benignware sets
#### Benign apks

In [82]:
%time top_bw_permissions = df_bw_permissions.sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_bw_permissions))
top_bw_permissions

CPU times: user 148 ms, sys: 56 ms, total: 204 ms
Wall time: 199 ms


PERMISSION-android.permission.INTERNET                  0.948182
PERMISSION-android.permission.ACCESS_NETWORK_STATE      0.729455
PERMISSION-android.permission.ACCESS_NETWORK_STATE.1    0.729455
PERMISSION-android.permission.WRITE_EXTERNAL_STORAGE    0.614909
PERMISSION-android.permission.WAKE_LOCK                 0.416000
PERMISSION-android.permission.ACCESS_WIFI_STATE.1       0.391364
PERMISSION-android.permission.ACCESS_WIFI_STATE         0.391364
PERMISSION-android.permission.READ_PHONE_STATE          0.371273
PERMISSION-android.permission.VIBRATE                   0.333273
PERMISSION-android.permission.ACCESS_FINE_LOCATION      0.277000
dtype: float64

#### Malware apks

In [80]:
%time top_mw_permissions = df_mw_permissions.sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_mw_permissions))
top_mw_permissions

CPU times: user 168 ms, sys: 48 ms, total: 216 ms
Wall time: 213 ms


PERMISSION-android.permission.INTERNET                  0.962091
PERMISSION-android.permission.ACCESS_NETWORK_STATE      0.854455
PERMISSION-android.permission.ACCESS_NETWORK_STATE.1    0.854455
PERMISSION-android.permission.WRITE_EXTERNAL_STORAGE    0.813091
PERMISSION-android.permission.READ_PHONE_STATE          0.802091
PERMISSION-android.permission.ACCESS_WIFI_STATE.1       0.604000
PERMISSION-android.permission.ACCESS_WIFI_STATE         0.604000
PERMISSION-android.permission.WAKE_LOCK                 0.490545
PERMISSION-android.permission.ACCESS_COARSE_LOCATION    0.419909
PERMISSION-android.permission.GET_TASKS                 0.391182
dtype: float64

## 2.3. System commands

In [7]:
#load_data
df_system = pd.read_csv("../data/interim/system_commands.csv")

In [8]:
df_system = df_system.iloc[:,1:]
df_system.head()

,label,SYSTEMCOMMAND-vold,SYSTEMCOMMAND-getevent,SYSTEMCOMMAND-setprop,SYSTEMCOMMAND-bugreport,SYSTEMCOMMAND-debuggerd,SYSTEMCOMMAND-sdcard,SYSTEMCOMMAND-chmod,SYSTEMCOMMAND-cd,SYSTEMCOMMAND-app_process,...,SYSTEMCOMMAND-svc,SYSTEMCOMMAND-run-as,SYSTEMCOMMAND-stagefright,SYSTEMCOMMAND-chown,SYSTEMCOMMAND-sh,SYSTEMCOMMAND-mediaserver,SYSTEMCOMMAND-gzip,SYSTEMCOMMAND-dexopt,SYSTEMCOMMAND-lsof,SYSTEMCOMMAND-nandread
0,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,MW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0


In [9]:
#split dataframe to benign apks and malware apks
df_bw_system = df_system[df_system["label"] == ''"BW"]
df_mw_system = df_system[df_system["label"] == "MW"]

### 10 most frequent system commands found in smali code obtained for each application in the malware and benignware sets
#### Benign apks

In [103]:
df_bw_system.describe()

,SYSTEMCOMMAND-vold,SYSTEMCOMMAND-getevent,SYSTEMCOMMAND-setprop,SYSTEMCOMMAND-bugreport,SYSTEMCOMMAND-debuggerd,SYSTEMCOMMAND-sdcard,SYSTEMCOMMAND-chmod,SYSTEMCOMMAND-cd,SYSTEMCOMMAND-app_process,SYSTEMCOMMAND-system_server,...,SYSTEMCOMMAND-svc,SYSTEMCOMMAND-run-as,SYSTEMCOMMAND-stagefright,SYSTEMCOMMAND-chown,SYSTEMCOMMAND-sh,SYSTEMCOMMAND-mediaserver,SYSTEMCOMMAND-gzip,SYSTEMCOMMAND-dexopt,SYSTEMCOMMAND-lsof,SYSTEMCOMMAND-nandread
count,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,...,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000
mean,0.005455,0.000091,0.001545,0.001273,0.001273,0.098818,0.134364,0.721000,0.003091,0.001000,...,0.003364,0.000455,0.000273,0.019182,0.873727,0.000545,2.857818,0.000364,0.000273,0.000091
std,0.081842,0.009535,0.063944,0.044705,0.094384,0.501981,1.191593,1.241564,0.166221,0.043684,...,0.097647,0.021316,0.016513,1.045860,2.518043,0.030148,5.617223,0.019067,0.016513,0.009535
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000
max,3.000000,1.000000,5.000000,2.000000,7.000000,23.000000,61.000000,28.000000,12.000000,3.000000,...,4.000000,1.000000,1.000000,102.000000,55.000000,2.000000,55.000000,1.000000,1.000000,1.000000


In [106]:
%time top_bw_system = df_bw_system.where(df_bw_system == 0, 1).sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_bw_system))
top_bw_system

CPU times: user 108 ms, sys: 24 ms, total: 132 ms
Wall time: 126 ms


SYSTEMCOMMAND-id         0.685636
SYSTEMCOMMAND-top        0.668273
SYSTEMCOMMAND-start      0.595545
SYSTEMCOMMAND-service    0.590364
SYSTEMCOMMAND-gzip       0.523455
SYSTEMCOMMAND-input      0.474091
SYSTEMCOMMAND-mv         0.468545
SYSTEMCOMMAND-log        0.434091
SYSTEMCOMMAND-sh         0.428909
SYSTEMCOMMAND-stop       0.373273
dtype: float64

#### malware apks

In [17]:
%time top_mw_system = df_mw_system.where(df_mw_system == 0, 1).sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_mw_system))
top_mw_system

CPU times: user 144 ms, sys: 8 ms, total: 152 ms
Wall time: 141 ms


SYSTEMCOMMAND-id         0.671364
SYSTEMCOMMAND-start      0.533818
SYSTEMCOMMAND-gzip       0.415182
SYSTEMCOMMAND-date       0.395545
SYSTEMCOMMAND-service    0.326091
SYSTEMCOMMAND-log        0.324818
SYSTEMCOMMAND-input      0.275909
SYSTEMCOMMAND-stop       0.266455
SYSTEMCOMMAND-sh         0.259091
SYSTEMCOMMAND-top        0.249818
dtype: float64

## 2.4. Receivers

In [12]:
# load data
df_receivers = pd.read_csv("../data/interim/intent_receivers.csv")
df_receivers = df_receivers.iloc[:,1:]

In [36]:
df_receivers.head()

,label,RECEIVER-,RECEIVER-it.samira.mylib.receivers.AudioPlayerReceiver_CONTROL,RECEIVER-com.igexin.sdk.action.cIRZ7PvDXY8zqOgEZSBGm1,RECEIVER-com.mobisleapps.maybe.like.description,RECEIVER-com.sec.everglades.main.update_for_widget,RECEIVER-org.mozilla.webapp.TASK_REMOVED,RECEIVER-action_show_10km,RECEIVER-com.cn21.android.pushservice.action.MESSAGE,RECEIVER-com.ganji.android.action.START_ALARM,...,RECEIVER-com.android.vending.INSTALL_REFERRER,RECEIVER-com.jb.gosms.im.IM_NEW_MESSAGE,RECEIVER-com.garena.android.gpns.NOTIFICATION_RECEIVE,RECEIVER-com.mymoney.sms.action.GET_CACHE_DATA_REMIND,RECEIVER-com.colormar.colorWeatherxdfghjwfwfghj.appWidget.ACTION_SHOW_MORE,RECEIVER-com.syezon.action.ADJUST_ORDER,RECEIVER-cn.jpush.android.intent.UNREGISTRATION,RECEIVER-com.appbyme.app64872.forum.service.heart.beat.notify,RECEIVER-com.optaim.zyzservice.SETUP,RECEIVER-android.intent.action.SEND_MESSAGE
0,BW,1,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,BW,1,0,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,BW,1,0,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,BW,1,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,MW,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
#split dataframe to benign apks and malware apks
df_bw_receivers = df_receivers[df_receivers["label"] == ''"BW"]
df_mw_receivers = df_receivers[df_receivers["label"] == "MW"]

### 10 most frequent receivers found in smali code obtained for each application in the malware and benignware sets
#### Benign apks

In [40]:
df_bw_receivers.describe()

,RECEIVER-,RECEIVER-it.samira.mylib.receivers.AudioPlayerReceiver_CONTROL,RECEIVER-com.igexin.sdk.action.cIRZ7PvDXY8zqOgEZSBGm1,RECEIVER-com.mobisleapps.maybe.like.description,RECEIVER-com.sec.everglades.main.update_for_widget,RECEIVER-org.mozilla.webapp.TASK_REMOVED,RECEIVER-action_show_10km,RECEIVER-com.cn21.android.pushservice.action.MESSAGE,RECEIVER-com.ganji.android.action.START_ALARM,RECEIVER-com.silentcom.osutils.ALARM_ALERT,...,RECEIVER-com.android.vending.INSTALL_REFERRER,RECEIVER-com.jb.gosms.im.IM_NEW_MESSAGE,RECEIVER-com.garena.android.gpns.NOTIFICATION_RECEIVE,RECEIVER-com.mymoney.sms.action.GET_CACHE_DATA_REMIND,RECEIVER-com.colormar.colorWeatherxdfghjwfwfghj.appWidget.ACTION_SHOW_MORE,RECEIVER-com.syezon.action.ADJUST_ORDER,RECEIVER-cn.jpush.android.intent.UNREGISTRATION,RECEIVER-com.appbyme.app64872.forum.service.heart.beat.notify,RECEIVER-com.optaim.zyzservice.SETUP,RECEIVER-android.intent.action.SEND_MESSAGE
count,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,...,11000.0,11000.0,11000.0,11000.0,11000.0,11000.0,11000.0,11000.0,11000.0,11000.0
mean,0.614636,0.058273,0.098364,0.013909,0.877091,0.007909,0.832636,0.951818,0.500182,0.965636,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.486703,0.234269,0.297819,0.117119,0.328348,0.088585,0.373317,0.214160,0.500023,0.182170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
%time top_bw_receivers = df_bw_receivers.sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_bw_receivers))
top_bw_receivers

CPU times: user 140 ms, sys: 0 ns, total: 140 ms
Wall time: 136 ms


RECEIVER-LAST                                                                             0.996545
RECEIVER-intent.close.sms                                                                 0.995636
RECEIVER-com.crowdcompass.com.crowdcompass.appdSXv7sDjIn.reminder.fire                    0.994364
RECEIVER-CLICK                                                                            0.990182
RECEIVER-DELETE_NOTIFICATION                                                              0.985727
RECEIVER-android.search.action.SETTINGS_CHANGED                                           0.966455
RECEIVER-com.silentcom.osutils.ALARM_ALERT                                                0.965636
RECEIVER-net.anei.cadpage.REQ_ACCOUNT_INFO.CodeMessaging                                  0.964727
RECEIVER-nd.panda.custom.widget.install.action                                            0.964727
RECEIVER-com.magplus.svenbenny.whitelabelapplication.intent.APPBOY_NOTIFICATION_OPENED    0.962818
dtype: flo

#### malware apks

In [45]:
%time top_mw_receivers = df_mw_receivers.sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_mw_receivers))
top_mw_receivers

CPU times: user 140 ms, sys: 0 ns, total: 140 ms
Wall time: 136 ms


RECEIVER-LAST                                                                             0.996545
RECEIVER-intent.close.sms                                                                 0.995636
RECEIVER-com.crowdcompass.com.crowdcompass.appdSXv7sDjIn.reminder.fire                    0.994364
RECEIVER-CLICK                                                                            0.990182
RECEIVER-DELETE_NOTIFICATION                                                              0.985727
RECEIVER-android.search.action.SETTINGS_CHANGED                                           0.966455
RECEIVER-com.silentcom.osutils.ALARM_ALERT                                                0.965636
RECEIVER-net.anei.cadpage.REQ_ACCOUNT_INFO.CodeMessaging                                  0.964727
RECEIVER-nd.panda.custom.widget.install.action                                            0.964727
RECEIVER-com.magplus.svenbenny.whitelabelapplication.intent.APPBOY_NOTIFICATION_OPENED    0.962818
dtype: flo

## 2.5. Services

In [13]:
#load data
df_services = pd.read_csv("../data/interim/intent_services.csv")
df_services = df_services.iloc[:, 1:]

In [47]:
df_services.head()

,label,SERVICE-,SERVICE-com.editorinc.greetingcards.getwellsoon.modules.podcast.download.ACTION_STOP_DOWNLOADS,SERVICE-com.guide.flippywheelsguide.modules.podcast.player.ACTION_TOGGLE_PLAY,SERVICE-com.skb.btvmobile.popuplayservice.UI_SHOW_RESIZEBTN,SERVICE-com.borya.cclingtoneconcall.action.START_CCS_SERVICE,SERVICE-cn.mchina.haokan.action.user_sign_in,SERVICE-android.service.dreams.DreamService,SERVICE-de.bright_side.brightsound.service.action.NEXT,SERVICE-com.baidu.android.pushservice.action.PUSH_SERVICE,...,SERVICE-com.andromo.dev64620.app254035.soundboard.action.SKIP,SERVICE-com.juanjo.frasesparaenamorarmujer.modules.podcast.player.ACTION_STOP,SERVICE-mw.ocgvfhdkbxbb.lqqudspk.m42c1f214.wb190f26e.NEEVLLAZBHGWL,SERVICE-CLICK_CHIZROID,SERVICE-android.intent.action.CDE.HDP.MOBILE,SERVICE-uk.co.kavcom.zss_start,SERVICE-com.baihe.intent.STOP_PUSH_SERVICE,SERVICE-com.dgsd.ddg.modules.podcast.download.ACTION_STOP_DOWNLOADS,SERVICE-com.vblast.xiialive.intent.action.PLAYBACK_START,SERVICE-musicalatina.com.modules.podcast.player.ACTION_PAUSE
0,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
df_services.describe()

,SERVICE-,SERVICE-com.editorinc.greetingcards.getwellsoon.modules.podcast.download.ACTION_STOP_DOWNLOADS,SERVICE-com.guide.flippywheelsguide.modules.podcast.player.ACTION_TOGGLE_PLAY,SERVICE-com.skb.btvmobile.popuplayservice.UI_SHOW_RESIZEBTN,SERVICE-com.borya.cclingtoneconcall.action.START_CCS_SERVICE,SERVICE-cn.mchina.haokan.action.user_sign_in,SERVICE-android.service.dreams.DreamService,SERVICE-de.bright_side.brightsound.service.action.NEXT,SERVICE-com.baidu.android.pushservice.action.PUSH_SERVICE,SERVICE-com.popularapp.periodcalendar.NotificationPillService,...,SERVICE-com.andromo.dev64620.app254035.soundboard.action.SKIP,SERVICE-com.juanjo.frasesparaenamorarmujer.modules.podcast.player.ACTION_STOP,SERVICE-mw.ocgvfhdkbxbb.lqqudspk.m42c1f214.wb190f26e.NEEVLLAZBHGWL,SERVICE-CLICK_CHIZROID,SERVICE-android.intent.action.CDE.HDP.MOBILE,SERVICE-uk.co.kavcom.zss_start,SERVICE-com.baihe.intent.STOP_PUSH_SERVICE,SERVICE-com.dgsd.ddg.modules.podcast.download.ACTION_STOP_DOWNLOADS,SERVICE-com.vblast.xiialive.intent.action.PLAYBACK_START,SERVICE-musicalatina.com.modules.podcast.player.ACTION_PAUSE
count,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,...,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
#split dataframe to benign apks and malware apks
df_bw_services = df_services[df_services["label"] == ''"BW"]
df_mw_services = df_services[df_services["label"] == "MW"]

### 10 most frequent services found in smali code obtained for each application in the malware and benignware sets
#### Benign apks

In [60]:
top_bw_services = df_bw_services.mean().sort_values(ascending=False)[:10]

top_bw_services

SERVICE-com.crowdcompass.bearing.client.navigator.LockedContentService          0.000909
SERVICE-com.mediatek.filemanager.service.FileManagerService                     0.000455
SERVICE-com.dynamixsoftware.printservice.PrintingService                        0.000273
SERVICE-cn.bmob.push.lib.service.PushService                                    0.000273
SERVICE-com.shoujiduoduo.wallpaper.autochange.WallpaperDuoduoService            0.000273
SERVICE-com.jb.gokeyboard.theme.template.alarm.GoKeyboardInstallCheckService    0.000182
SERVICE-org.windbell.window.QuickHeplerFloatWindow                              0.000182
SERVICE-com.videogo.androidpn.NotificationService                               0.000182
SERVICE-com.videogo.androidpn.GCMService                                        0.000182
SERVICE-com.ev123.service.MainService                                           0.000182
dtype: float64

#### Malware apks

In [61]:
top_mw_services = df_mw_services.mean().sort_values(ascending=False)[:10]

top_mw_services

SERVICE-com.snowfish.a.a.s.ABGSvc               0.004727
SERVICE-com.android.jservice.UpdateService      0.004273
SERVICE-jp.bravo.honda.FServices                0.002182
SERVICE-com.iadpush.adp.NS                      0.001818
SERVICE-ru.mskdev.andrinst.NewsReaderService    0.001818
SERVICE-com.iadpush.adp.BS                      0.001818
SERVICE-com.PushService.android.PushService     0.001636
SERVICE-com.bx.pay.WpaySmsService               0.001636
SERVICE-cn.zhui.push.TuisongService             0.001455
SERVICE-com.zhuaz.util.MMobanService            0.001364
dtype: float64

## 2.6. Activities

In [14]:
#load data
df_activities = pd.read_csv("../data/interim/intent_activities.csv")
df_activities = df_activities.iloc[:, 1:]

In [63]:
df_activities.head()
df_activities.describe()

,ACTIVITY-,ACTIVITY-com.hanmiOnlinePharm.sfa.ui.threecall.VisitCallProgressActivity,ACTIVITY-android.intent.action.mamafu.INTERMEDIATE,ACTIVITY-com.fusepowered.fuseactivities.FuseApiMoregamesBrowser,ACTIVITY-io.rong.voipkit.calledSideActivity.action,ACTIVITY-android.intent.action.Run,ACTIVITY-com.quizmine.androidnutrition.tutorialActivity_slab,ACTIVITY-seven.client.ntr.android.QR.SCAN,ACTIVITY-com.experlabs.timepass.PREFS,ACTIVITY-qgl_seoul.ACTION_MULTIPLE_PICK,...,ACTIVITY-com.sonyericsson.extras.liveware.SHOW_CUSTOMIZED_MSG,ACTIVITY-com.myingzhijia.CustomApplyServiceActivity,ACTIVITY-k.microcells.running.GameActivity,ACTIVITY-com.quizmine.thirdrdgradeccmath.howToUseVideoActivity,ACTIVITY-com.meituan.android.intent.action.address_edit_meituan_activity,ACTIVITY-android.intent.action.cpsl.checkoutuld,ACTIVITY-action.cidian.search,ACTIVITY-com.kukool.iphonestock.activity.AddCity,ACTIVITY-mituo.plat.intent.action.ADS.NOTIFIER.CLICKED,ACTIVITY-com.fresvii.chickenballoon.NotificationAction
count,22000.000000,22000.000000,22000.0,22000.000000,22000.0,22000.0,22000.0,22000.0,22000.0,22000.0,...,22000.0,22000.000000,22000.000000,22000.0,22000.0,22000.0,22000.0,22000.000000,22000.0,22000.0
mean,0.000045,0.000045,0.0,0.000318,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000045,0.000045,0.0,0.0,0.0,0.0,0.000045,0.0,0.0
std,0.006742,0.006742,0.0,0.017835,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.006742,0.006742,0.0,0.0,0.0,0.0,0.006742,0.0,0.0
min,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
25%,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
50%,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
75%,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
max,1.000000,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.000000,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0


In [65]:
#split dataframe to benign apks and malware apks
df_bw_activities = df_activities[df_activities["label"] == ''"BW"]
df_mw_activities = df_activities[df_activities["label"] == "MW"]

### 10 most frequent activities found in smali code obtained for each application in the malware and benignware sets
#### Benign apks

In [66]:
top_bw_activities = df_bw_activities.mean().sort_values(ascending=False)[:10]

top_bw_activities

ACTIVITY-cn.jpush.android.ui.PushActivity                           0.008818
ACTIVITY-com.doapps.android.mln.settings.AboutActivity              0.001455
ACTIVITY-com.doapps.android.mln.settings.SettingsActivity           0.001455
ACTIVITY-com.mystylinglounge.salongame.MainActivity                 0.000455
ACTIVITY-com.fusepowered.fuseactivities.FuseApiMoregamesBrowser     0.000455
ACTIVITY-com.tremorvideo.sdk.android.videoad.Playvideo              0.000364
ACTIVITY-com.mediatek.FMRadio.FMRadioEMActivity                     0.000364
ACTIVITY-com.ixintui.push.PushActivity                              0.000182
ACTIVITY-com.android.inputmethod.latin.settings.SettingsActivity    0.000182
ACTIVITY-com.smustafa.praytimes.PrayTimes                           0.000182
dtype: float64

#### Malware apks

In [67]:
top_mw_activities = df_mw_activities.mean().sort_values(ascending=False)[:10]

top_mw_activities

ACTIVITY-cn.jpush.android.ui.PushActivity                         0.015636
ACTIVITY-com.shaziguazi.util.MyFormActivity                       0.003727
ACTIVITY-com.shaziguazi.util.Dialog2Activity                      0.003727
ACTIVITY-com.shaziguazi.util.Dialog1_3Activity                    0.003727
ACTIVITY-com.shaziguazi.third.ThirdDialogActivity                 0.003727
ACTIVITY-com.shaziguazi.util.ShowTipsActivity                     0.003727
ACTIVITY-com.nd.commplatform.activity.SNSControlCenterActivity    0.001182
ACTIVITY-com.dark.util.Dialog2Activity                            0.000182
ACTIVITY-com.dark.util.Dialog1_3Activity                          0.000182
ACTIVITY-com.anrd.sysservices.InstallActivity                     0.000182
dtype: float64

## 2.7. API calls

In [15]:
#load data
df_api_calls = pd.read_csv("../data/interim/api_calls.csv")
df_api_calls = df_api_calls.iloc[:,1:]
df_api_calls.head()
df_api_calls.describe()

,APICALL-java.security.cert.Certificate,APICALL-android.view.SubMenu,APICALL-android.media.audiofx.Virtualizer,APICALL-android.support.v7.widget.helper.ItemTouchHelper,APICALL-android.net.RouteInfo,APICALL-android.bluetooth.BluetoothGattCallback,APICALL-android.support.percent.PercentLayoutHelper,APICALL-android.print.PageRange,APICALL-java.net.InterfaceAddress,APICALL-android.support.v7.widget.DividerItemDecoration,...,APICALL-android.speech,APICALL-android.content.ClipDescription,APICALL-android.support.v4.view.accessibility.AccessibilityManagerCompat,APICALL-java.net.InetAddress,APICALL-android.support.multidex.MultiDexApplication,APICALL-android.widget.MediaController,APICALL-android.support.v4.widget.ExploreByTouchHelper,APICALL-android.content.ContentResolver,APICALL-org.xmlpull.v1.XmlPullParser,APICALL-org.xml.sax.ext.DeclHandler
count,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.00000,22000.000000,22000.000000,...,22000.0,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000
mean,0.292545,2.596636,0.005591,5.097636,0.002636,0.003182,0.030364,0.00200,0.008500,0.001227,...,0.0,0.635682,1.367000,3.766773,0.025727,1.629273,12.803818,12.108273,31.229045,0.036455
std,2.113503,4.354389,0.230056,21.344695,0.179884,0.151927,0.601365,0.13617,0.222124,0.060667,...,0.0,0.950356,1.499259,8.454665,0.230167,4.756713,15.146042,25.144656,92.923261,0.383697
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,7.000000,9.000000,0.000000
75%,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,2.000000,3.000000,4.000000,0.000000,1.000000,30.000000,14.000000,16.000000,0.000000
max,50.000000,58.000000,16.000000,101.000000,13.000000,12.000000,12.000000,18.00000,19.000000,3.000000,...,0.0,15.000000,65.000000,212.000000,5.000000,102.000000,62.000000,1015.000000,2014.000000,10.000000


In [16]:
#split dataframe to benign apks and malware apks
df_bw_api_calls = df_api_calls[df_api_calls["label"] == ''"BW"]
df_mw_api_calls = df_api_calls[df_api_calls["label"] == "MW"]

### 10 most frequent activities found in smali code obtained for each application in the malware and benignware sets

#### benign apks

In [72]:
%time top_bw_api_calls = df_bw_api_calls.where(df_bw_api_calls == 0, 1).sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_bw_api_calls))
top_bw_api_calls

CPU times: user 1.98 s, sys: 264 ms, total: 2.24 s
Wall time: 2.24 s


APICALL-java.lang.StringBuilder          0.993909
APICALL-java.lang.String                 0.991818
APICALL-android.content.Context          0.988636
APICALL-java.lang.System                 0.984545
APICALL-android.content.res.Resources    0.982545
APICALL-java.io.File                     0.980000
APICALL-java.lang.Boolean                0.973455
APICALL-android.app.Activity             0.972182
APICALL-java.io.InputStream              0.966273
APICALL-java.lang.Integer                0.965273
dtype: float64

#### malware apks

In [73]:
%time top_mw_api_calls = df_mw_api_calls.where(df_mw_api_calls == 0, 1).sum(numeric_only=True).sort_values(ascending=False)[:10].apply(lambda x: x/len(df_mw_api_calls))
top_mw_api_calls

CPU times: user 4.89 s, sys: 1.35 s, total: 6.24 s
Wall time: 6.34 s


APICALL-java.lang.String           0.986636
APICALL-java.lang.StringBuilder    0.985909
APICALL-android.content.Context    0.962909
APICALL-java.lang.System           0.962091
APICALL-android.app.Activity       0.939273
APICALL-java.lang.Integer          0.930091
APICALL-android.content.Intent     0.922727
APICALL-android.net.Uri            0.907636
APICALL-android.util.Log           0.902636
APICALL-java.io.InputStream        0.900091
dtype: float64

# TODO: 3. Feature Engineering

# 4. Preprocessing data & Modeling

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import accuracy_score

In [ ]:
%time small_df = pd.read_csv("../data/interim/test/test_df.csv")

In [ ]:
small_df.drop(list(df.filter(regex = '^APIPACKAGE')), axis = 1, inplace = True)

In [ ]:
df_train, df_test = train_test_split(small_df, test_size=None, train_size=0.7, random_state = 3)

In [28]:
df_train = pd.get_dummies(df_train, columns=["label"], drop_first=True, prefix='', prefix_sep='')
df_test = pd.get_dummies(df_test, columns=["label"], drop_first=True, prefix='', prefix_sep='')

In [29]:
df_train.head(1) # label -> MW

,PERMISSION-com.amazon.CONTENT_PROVIDER_ACCESS,PERMISSION-com.mducrad.taskmanagerapp.permission.C2D_MESSAGE,PERMISSION-com.mywavia.poker.android.permission.C2D_MESSAGE,PERMISSION-android.permission.INTERNET,PERMISSION-com.yandex.promolib.permission.ACCESS_START_SVC,PERMISSION-com.android.buycott.permission.C2D_MESSAGE,PERMISSION-com.triactivemedia.familytree.permission.C2D_MESSAGE,PERMISSION-air.hdandv.hashgab.permission.C2D_MESSAGE,PERMISSION-com.Settings.widget.permission.SettingsAppWidgetProvider,PERMISSION-com.ionicframework.codeartfilmnews445890.permission.C2D_MESSAGE,...,RECEIVER-com.jb.gosms.im.IM_NEW_MESSAGE,RECEIVER-com.garena.android.gpns.NOTIFICATION_RECEIVE,RECEIVER-com.mymoney.sms.action.GET_CACHE_DATA_REMIND,RECEIVER-com.colormar.colorWeatherxdfghjwfwfghj.appWidget.ACTION_SHOW_MORE,RECEIVER-com.syezon.action.ADJUST_ORDER,RECEIVER-cn.jpush.android.intent.UNREGISTRATION,RECEIVER-com.appbyme.app64872.forum.service.heart.beat.notify,RECEIVER-com.optaim.zyzservice.SETUP,RECEIVER-android.intent.action.SEND_MESSAGE,MW
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# for df_train_ml
scaler.fit(df_train.drop(['MW'],axis=1))
scaled_features = scaler.transform(df_train.drop(['MW'],axis=1))
df_train_sc = pd.DataFrame(scaled_features) # columns=df_train_ml.columns[1::])


# scaler.fit(df_test_ml)
scaled_features = scaler.transform(df_test.drop(['MW'],axis=1))
df_test_sc = pd.DataFrame(scaled_features) # , columns=df_test_ml.columns)

#### load data

In [3]:
%time df_train = pd.read_csv("../data/processed/test/train.csv")
%time df_test  = pd.read_csv("../data/processed/test/test.csv")

df_train.head(1)
# df_train = df_train.iloc[:,1:]
# df_test = df_test.iloc[:, 1:]

CPU times: user 2min 15s, sys: 3.79 s, total: 2min 18s
Wall time: 2min 19s
CPU times: user 1min 1s, sys: 996 ms, total: 1min 2s
Wall time: 1min 25s


,PERMISSION-com.amazon.CONTENT_PROVIDER_ACCESS,PERMISSION-com.mducrad.taskmanagerapp.permission.C2D_MESSAGE,PERMISSION-com.mywavia.poker.android.permission.C2D_MESSAGE,PERMISSION-android.permission.INTERNET,PERMISSION-com.yandex.promolib.permission.ACCESS_START_SVC,PERMISSION-com.android.buycott.permission.C2D_MESSAGE,PERMISSION-com.triactivemedia.familytree.permission.C2D_MESSAGE,PERMISSION-air.hdandv.hashgab.permission.C2D_MESSAGE,PERMISSION-com.Settings.widget.permission.SettingsAppWidgetProvider,PERMISSION-com.ionicframework.codeartfilmnews445890.permission.C2D_MESSAGE,...,RECEIVER-com.jb.gosms.im.IM_NEW_MESSAGE,RECEIVER-com.garena.android.gpns.NOTIFICATION_RECEIVE,RECEIVER-com.mymoney.sms.action.GET_CACHE_DATA_REMIND,RECEIVER-com.colormar.colorWeatherxdfghjwfwfghj.appWidget.ACTION_SHOW_MORE,RECEIVER-com.syezon.action.ADJUST_ORDER,RECEIVER-cn.jpush.android.intent.UNREGISTRATION,RECEIVER-com.appbyme.app64872.forum.service.heart.beat.notify,RECEIVER-com.optaim.zyzservice.SETUP,RECEIVER-android.intent.action.SEND_MESSAGE,MW
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_train_sc = pd.read_csv("../data/processed/test/train_sc.csv")
df_test_sc = pd.read_csv("../data/processed/test/test_sc.csv")

df_train_sc.head(2)

,0,1,2,3,4,5,6,7,8,9,...,24815,24816,24817,24818,24819,24820,24821,24822,24823,24824
0,0.0,0.0,0.0,0.219846,0.0,0.0,0.0,0.0,-0.012742,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.219846,0.0,0.0,0.0,0.0,-0.012742,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X = df_train.drop('MW', axis=1)
y = df_train['MW']
X_test = df_test

X_sc = df_train_sc
y_sc = df_train['MW']
X_test_sc = df_test_sc

#### save data 

In [33]:
%time df_train_sc.to_csv('../data/processed/test/train_sc.csv', index=False)
%time df_test_sc.to_csv('../data/processed/test/test_sc.csv', index=False)

%time df_train.to_csv("../data/processed/test/train.csv", index=False)
%time df_test.to_csv("../data/processed/test/test.csv", index=False)

CPU times: user 4min 32s, sys: 11.1 s, total: 4min 44s
Wall time: 4min 44s
CPU times: user 1min 57s, sys: 4.86 s, total: 2min 2s
Wall time: 2min 2s
CPU times: user 5min 4s, sys: 9.56 s, total: 5min 13s
Wall time: 5min 14s
CPU times: user 2min 10s, sys: 4.42 s, total: 2min 14s
Wall time: 2min 15s


#### Train model

In [6]:
from sklearn.model_selection import cross_val_score
cross_validation_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 )

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [24]:
# RFC Parameters tunning 

RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=cross_validation_split, scoring="accuracy", n_jobs= 8, verbose = 10)

gsRFC.fit(X_sc,y_sc)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   22.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   38.3s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  2.1min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  2.5min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  2.9min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  3.2min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  3.4min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  3.5min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  3.7min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  3.9min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  4.0min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  4.4min
[Parallel(

0.8527597402597402

In [8]:
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=cross_validation_split, scoring="accuracy", n_jobs= 8, verbose = 10)

gsSVMC.fit(X_sc,y_sc)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

Fitting 10 folds for each of 28 candidates, totalling 280 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed: 77.5min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed: 163.0min


KeyboardInterrupt: 

In [9]:
import xgboost as xg

In [10]:
from sklearn.model_selection import GridSearchCV
n_estimators=list(range(100,1100,100))
learn_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
gd_xgb=GridSearchCV(estimator=xg.XGBClassifier(),param_grid=hyper,verbose=True, n_jobs=8, cv=cross_validation_split)
gd_xgb.fit(X,y)
print(gd_xgb.best_score_)
print(gd_xgb.best_estimator_)

Fitting 10 folds for each of 120 candidates, totalling 1200 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators=list(range(100,1100,100))
learn_rate=[0.01,0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
gd_ada=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=hyper,verbose=True, n_jobs=8, cv=cross_validation_split)
gd_ada.fit(X,y)
print(gd_ada.best_score_)
print(gd_ada.best_estimator_)

Fitting 10 folds for each of 130 candidates, totalling 1300 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
